In [ ]:
### Import necessary depencencies
import multiprocessing
import pickle
from multiprocessing import Process
import numpy as np
import pandas as pd
import utils

PROCESSED_FILENAME_A= './data/amazon_reviews_processed_partA.pickle' 
PROCESSED_FILENAME_B= './data/amazon_reviews_processed_partB.pickle' 
trial=0
#number of processes to run in multiprocessing
processors=16

pd.set_option('display.max_colwidth', -1)

# ============================================
#   Preprocess reviews 
# ============================================


In [ ]:
def readFromDisk():
    with open(PROCESSED_FILENAME_A, "rb") as f:
        dfread = pickle.load(f)
        return dfread
        
def writeToDisk(dfdb):
    with open(PROCESSED_FILENAME_B, "wb") as f:
        pickle.dump(dfdb, f)
        
        
        

In [ ]:
#nltk.download('all', halt_on_error = True)

df=readFromDisk()

# FOR TESTING
#df=df.iloc[0:1000]


#number of observations
size=len(df)
#number of observations to normalize in each process
iterSize=round(size/processors)
#holds the processes
processes=[]

print('To process: ' + str(size))



In [ ]:
#dfshow=df[['reviewText','summary','Clean_Review','Clean_Review_Tokens']]
#dfshow=df['reviewText']
#dfshow

df.head(10)

In [ ]:
i=0

dfholder=[]
parent_conn_holder=[]

columns2=df.columns.values.tolist()
columns2.append('Clean_Review')
columns2.append('Clean_Review_Tokens')

dffile=pd.DataFrame(columns=columns2);


for i in range(0,processors):
    start=i*iterSize
    stop=start+iterSize
    if(stop>size):
        stop=size
    #split df for parallel proc
    dflist=df.iloc[start:stop]
    # creating a pipe 
    parent_conn, child_conn = multiprocessing.Pipe() 
    p = Process(target=utils.multiprocNormalize, args=(dflist,child_conn,"proc"+str(i)))
    #dfholder.append(utils.multiprocNormalize(dflist,"proc"+str(i)))
    processes.append(p)
    p.start()
    parent_conn_holder.append(parent_conn)

 
for parent_conn in parent_conn_holder:
    dfdb=parent_conn.recv()
    #take out leading and trailing blanks
    dfdb['Clean_Review'] = dfdb['Clean_Review'].str.strip().astype(str)
    #replace blank ones with nan
    dfdb['Clean_Review'].replace('', np.NaN, inplace=True)
    # drop nans
    dfdb.dropna(subset=['brand','reviewText','asin','overall','sentiment','Clean_Review'], inplace=True)
    dfdb=dfdb.reset_index(drop=True)
    dffile=pd.concat([dffile, dfdb], axis=0)

for p in processes:
    p.join()


In [ ]:
print('Total Rows on processed dataset: ' + str(len(dffile)))
print('Sample of processed dataset. Notice the column named Clean_Review');
dffile.head(10)

In [ ]:
writeToDisk(dffile)


In [ ]:
reviews = np.array(dffile['Clean_Review'])
reviews_raw = np.array(dffile['reviewText'])
sentiments = np.array(dffile['sentiment'])
reviews_tokens = np.array(dffile['Clean_Review_Tokens'])

reviews_positive = np.array( (dffile[dffile['sentiment'] == 1 ])['Clean_Review'])
reviews_negative = np.array( (dffile[dffile['sentiment'] == 0 ])['Clean_Review'])
reviews_positive_tokens = np.array( (dffile[dffile['sentiment'] == 1 ])['Clean_Review_Tokens'])
reviews_negative_tokens = np.array( (dffile[dffile['sentiment'] == 0 ])['Clean_Review_Tokens'])

utils.writeToDisk(reviews,'reviews')
utils.writeToDisk(reviews_raw,'reviews_raw')
utils.writeToDisk(sentiments,'sentiments')
utils.writeToDisk(reviews_tokens,'reviews_tokens')

utils.writeToDisk(reviews_positive,'reviews_positive')
utils.writeToDisk(reviews_negative,'reviews_negative')
utils.writeToDisk(reviews_positive_tokens,'reviews_positive_tokens')
utils.writeToDisk(reviews_negative_tokens,'reviews_negative_tokens')

In [ ]:
print('done')

    